In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
from google.cloud import bigquery
from bq_helper import BigQueryHelper

In [ ]:
#pip install scikit-learn-intelex
#pip install google-cloud-bigquery-storage
#install when running the book for the first time 


pip install scikit-learn-intelex

**pip install scikit-learn-intelex**



**pip install google-cloud-bigquery-storage**

In [ ]:
bq_assistant = BigQueryHelper("bigquery-public-data", "epa_historical_air_quality")

In [ ]:
QUERY = """
SELECT
    *
FROM
    `bigquery-public-data.ml_datasets.census_adult_income`
        """

In [ ]:
df = bq_assistant.query_to_pandas(QUERY)

df_census=df


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# sns.set_style('dark')
#plt.style.use('dark_background')
#plt.style.use('ggplot')

sns.countplot(y='occupation', hue='income_bracket', data=df_census)

# Count of >50K & <=50K
sns.countplot(df_census['income_bracket'],label="Count")

plt.figure(figsize=(10,8))
sns.countplot(y='education', hue='income_bracket', data=df_census)

In [ ]:

df = bq_assistant.query_to_pandas(QUERY)

df_census=df

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# sns.set_style('dark')
#plt.style.use('dark_background')
#plt.style.use('ggplot')

sns.countplot(y='occupation', hue='income_bracket', data=df_census)

# Count of >50K & <=50K
sns.countplot(df_census['income_bracket'],label="Count")

plt.figure(figsize=(10,8))
sns.countplot(y='education', hue='income_bracket', data=df_census)

In [ ]:
numeric_columns = df_census.select_dtypes(['int64']).columns
numeric_columns
# Correlation matrix between numerical values
sns.heatmap(df_census[numeric_columns].corr(),annot=True, fmt = ".2f", cmap = "coolwarm")

# Explore Age vs Income
g = sns.FacetGrid(df_census, col='income_bracket')
g = g.map(sns.distplot, "age")

df_census_sex=df_census['sex']
df_census_sex.head()
from sklearn.preprocessing import LabelEncoder
gender = LabelEncoder()
gender.fit(df_census_sex)
print(gender.classes_)

print(gender.transform([
    ' Female', ' Female', ' Male', ' Female', ' Male'
]))



In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
class EncodeCategorical(BaseEstimator, TransformerMixin):
    """
    Encodes a specified list of columns or all columns if None.
    """
    def __init__(self, columns=None):
        self.columns  = columns
        self.encoders = None
    def fit(self, data, target=None):
        """
        Expects a data frame with named columns to encode.
        """
        # Encode all columns if columns is None
        if self.columns is None:
            self.columns = data.columns
        # Fit a label encoder for each column in the data frame
        self.encoders = {
            column: LabelEncoder().fit(data[column])
            for column in self.columns
        }
        return self
    def transform(self, data):
        """
        Uses the encoders to transform a data frame.
        """
        output = data.copy()
        for column, encoder in self.encoders.items():
            output[column] = encoder.transform(data[column])
        return output



In [ ]:
numeric_columns = df_census.select_dtypes(['int64']).columns
numeric_columns

df_census[numeric_columns] = df_census[numeric_columns].astype('float32')

cat_columns = df_census.select_dtypes(['object']).columns
cat_columns


In [ ]:
test = LabelEncoder()
test.fit(df_census[cat_columns[0]])
print(test.classes_)

test.fit(df_census[cat_columns[1]])
print(test.classes_)

test.fit(df_census[cat_columns[2]])
print(test.classes_)

test.fit(df_census[cat_columns[3]])
print(test.classes_)

test.fit(df_census[cat_columns[4]])
print(test.classes_)

test.fit(df_census[cat_columns[5]])
print(test.classes_)

test.fit(df_census[cat_columns[7]])
print(test.classes_)

In [ ]:

encoder = EncodeCategorical(cat_columns)
df_census_encoded = encoder.fit_transform(df_census)
df_census_encoded.head()

from sklearn.impute import SimpleImputer
#https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html

class ImputeCategorical(BaseEstimator, TransformerMixin):
    """
    Encodes a specified list of columns or all columns if None.
    """
    def __init__(self, columns=None):
        self.columns = columns
        self.imputer = None
    def fit(self, data, target=None):
        """
        Expects a data frame with named columns to impute.
        """
        # Encode all columns if columns is None
        if self.columns is None:
            self.columns = data.columns
        # Fit an imputer for each column in the data frame
        self.imputer = SimpleImputer(missing_values=0, strategy='most_frequent')
        self.imputer.fit(data[self.columns])
        return self
    def transform(self, data):
        """
        Uses the encoders to transform a data frame.
        """
        output = data.copy()
        output[self.columns] = self.imputer.transform(output[self.columns])
        return output


In [ ]:
imputer = ImputeCategorical(['workclass', 'native_country', 'occupation'])
df_census_encoded = imputer.fit_transform(df_census_encoded)

df_census_encoded.head(50)

target=df_census_encoded['income_bracket']
target.head()

df_census_encoded=df_census_encoded.drop(columns=['income_bracket'])
df_census_encoded.head()


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df_census_encoded, target, train_size = 0.7, random_state=33)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

In [ ]:
from time import time
from sklearn import metrics


In [ ]:
start_unopt = time()
from sklearn.linear_model import LogisticRegression
lgreg=LogisticRegression(max_iter=200)
lgreg.fit(X_train, Y_train)
finish_unopt = time()
f"Execution time without Intel(R) Extension for Scikit-learn: {(finish_unopt - start_unopt):.2f} s"




In [ ]:
y_pred=lgreg.predict(X_test)



In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
print("roc_auc_score: ", roc_auc_score(Y_test, y_pred))
print("f1 score: ", f1_score(Y_test, y_pred))

In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()

In [ ]:
start_opt = time()
from sklearn.linear_model import LogisticRegression
lgreg=LogisticRegression(max_iter=200)
lgreg.fit(X_train, Y_train)
finish_opt = time()
f"Execution time with Intel(R) Extension for Scikit-learn: {(finish_opt - start_opt):.2f} s"

In [ ]:
y_pred=lgreg.predict(X_test)


In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
print("roc_auc_score: ", roc_auc_score(Y_test, y_pred))
print("f1 score: ", f1_score(Y_test, y_pred))

In [ ]:
import matplotlib.pyplot as plt
left = [1,2]
pred_times = [finish_opt - start_opt, finish_unopt - start_unopt]
tick_label = ['Intel Extension for SKLearn', 'Unoptimized SKLearn']
plt.bar(left, pred_times, tick_label = tick_label, width = 0.5, color = ['blue', 'red'])
plt.xlabel('Training Method'); plt.ylabel('time,s'); plt.title('Logistic Regression Training Time before standardizations')
plt.show()
print("Performance Speedup: ",(finish_unopt - start_unopt)/(finish_opt - start_opt))

In [ ]:
from sklearnex import unpatch_sklearn
unpatch_sklearn()

In [ ]:
X_train_c=X_train.copy()

# **STANDARDSCALER**

In [ ]:
from sklearn.preprocessing import StandardScaler

numeric_features = X_train_c[numeric_columns]
scaler = StandardScaler().fit(numeric_features.values)
trans_features = scaler.transform(numeric_features.values)
trans_features

In [ ]:
trans_features.shape

In [ ]:
X_train_c[numeric_columns] = trans_features
X_train_c

In [ ]:
X_test.head(20)

In [ ]:
X_test_c=X_test.copy()
numeric_features = X_test_c[numeric_columns]
scaler = StandardScaler().fit(numeric_features.values) 
trans_features = scaler.transform(numeric_features.values)
X_test_c[numeric_columns] = trans_features
X_test_c

# Post StandardScaler, Non Optimized Scikit Learn

In [ ]:
from sklearn.metrics import accuracy_score
start_unopt_log = time()
logreg = LogisticRegression(max_iter=300)
logreg.fit(X_train_c, Y_train)
finish_unopt_log = time()
final_unopt_log=(finish_unopt_log - start_unopt_log)
final_unopt_log=format(final_unopt_log, ".3f")
print(f"Execution time of Logistic Regression without Intel(R) Extension for Scikit-learn: {final_unopt_log} s \n")

y_pred = logreg.predict(X_test_c)

print('Logistic Regression accuracy score with all the features: {0:0.4f}'. format(accuracy_score(Y_test, y_pred)))
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred))
print("roc_auc_score: ", roc_auc_score(Y_test, y_pred))
print("f1 score: ", f1_score(Y_test, y_pred))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
start_unopt_knn = time()
knclassifier=KNeighborsClassifier()
knclassifier.fit(X_train_c, Y_train)
finish_unopt_knn = time()
final_unopt_knn=(finish_unopt_knn - start_unopt_knn)
final_unopt_knn=format(final_unopt_knn, ".3f")
print(f"Execution time of KNeighborsClassifier without Intel(R) Extension for Scikit-learn: {final_unopt_knn} s \n")
y_pred = knclassifier.predict(X_test_c)

print('KNeighborsClassifier accuracy score with all the features: {0:0.4f}'. format(accuracy_score(Y_test, y_pred)))
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred))
print("roc_auc_score: ", roc_auc_score(Y_test, y_pred))
print("f1 score: ", f1_score(Y_test, y_pred))

In [ ]:
from sklearn.svm import SVC
start_unopt_svc = time()
svcclassifier=SVC(kernel="linear", C=0.025)
svcclassifier.fit(X_train_c, Y_train)
finish_unopt_svc = time()
final_unopt_svc=(finish_unopt_svc - start_unopt_svc)
final_unopt_svc=format(final_unopt_svc, ".3f")
print(f"Execution time of SVC Classifier without Intel(R) Extension for Scikit-learn: {final_unopt_svc} s \n")
y_pred = svcclassifier.predict(X_test_c)

print('SVC Classifier accuracy score with all the features: {0:0.4f}'. format(accuracy_score(Y_test, y_pred)))
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred))
print("roc_auc_score: ", roc_auc_score(Y_test, y_pred))
print("f1 score: ", f1_score(Y_test, y_pred))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
start_unopt_dtc = time()
decclassifier=DecisionTreeClassifier(max_depth=5)
decclassifier.fit(X_train_c, Y_train)
finish_unopt_dtc = time()
final_unopt_dtc=(finish_unopt_dtc - start_unopt_dtc)
final_unopt_dtc=format(final_unopt_dtc, ".3f")
print(f"Execution time of Decision Tree Classifier without Intel(R) Extension for Scikit-learn: {final_unopt_dtc} s \n")
y_pred = decclassifier.predict(X_test_c)

print('Decision Tree Classifier accuracy score with all the features: {0:0.4f}'. format(accuracy_score(Y_test, y_pred)))
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred))
print("roc_auc_score: ", roc_auc_score(Y_test, y_pred))
print("f1 score: ", f1_score(Y_test, y_pred))

In [ ]:
unopt_times=[final_unopt_log,final_unopt_knn,final_unopt_svc,final_unopt_dtc]

In [ ]:
unopt_times

# Accelerated Intel Scikit Learn

In [ ]:
patch_sklearn()

In [ ]:
from sklearn.metrics import accuracy_score
start_opt_log = time()
logreg = LogisticRegression(max_iter=300)
logreg.fit(X_train_c, Y_train)
finish_opt_log = time()
final_opt_log=(finish_opt_log - start_opt_log)
final_opt_log=format(final_opt_log, ".3f")
print(f"Execution time of Logistic Regression with Intel(R) Extension for Scikit-learn: {final_opt_log} s \n")

y_pred = logreg.predict(X_test_c)

print('Logistic Regression accuracy score with all the features: {0:0.4f}'. format(accuracy_score(Y_test, y_pred)))
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred))
print("roc_auc_score: ", roc_auc_score(Y_test, y_pred))
print("f1 score: ", f1_score(Y_test, y_pred))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
start_opt_knn = time()
knclassifier=KNeighborsClassifier()
knclassifier.fit(X_train_c, Y_train)
finish_opt_knn = time()
final_opt_knn=(finish_opt_knn - start_opt_knn)
final_opt_knn=format(final_opt_knn, ".3f")
print(f"Execution time of KNeighborsClassifier with Intel(R) Extension for Scikit-learn: {final_opt_knn} s \n")
y_pred = knclassifier.predict(X_test_c)

print('KNeighborsClassifier accuracy score with all the features: {0:0.4f}'. format(accuracy_score(Y_test, y_pred)))
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred))
print("roc_auc_score: ", roc_auc_score(Y_test, y_pred))
print("f1 score: ", f1_score(Y_test, y_pred))

In [ ]:
from sklearn.svm import SVC
start_opt_svc = time()
svcclassifier=SVC(kernel="linear", C=0.025)
svcclassifier.fit(X_train_c, Y_train)
finish_opt_svc = time()
final_opt_svc=(finish_opt_svc - start_opt_svc)
final_opt_svc=format(final_opt_svc, ".3f")
print(f"Execution time of SVC Classifier with Intel(R) Extension for Scikit-learn: {final_opt_svc} s \n")
y_pred = svcclassifier.predict(X_test_c)

print('SVC Classifier accuracy score with all the features: {0:0.4f}'. format(accuracy_score(Y_test, y_pred)))
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred))
print("roc_auc_score: ", roc_auc_score(Y_test, y_pred))
print("f1 score: ", f1_score(Y_test, y_pred))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
start_opt_dtc = time()
decclassifier=DecisionTreeClassifier(max_depth=5)
decclassifier.fit(X_train_c, Y_train)
finish_opt_dtc = time()
final_opt_dtc=(finish_opt_dtc - start_opt_dtc)
final_opt_dtc=format(final_opt_dtc, ".3f")
print(f"Execution time of Decision Tree Classifier with Intel(R) Extension for Scikit-learn: {final_opt_dtc} s \n")
y_pred = decclassifier.predict(X_test_c)

print('Decision Tree Classifier accuracy score with all the features: {0:0.4f}'. format(accuracy_score(Y_test, y_pred)))
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred))
print("roc_auc_score: ", roc_auc_score(Y_test, y_pred))
print("f1 score: ", f1_score(Y_test, y_pred))

In [ ]:
opt_times=[final_opt_log,final_opt_knn,final_opt_svc,final_opt_dtc]

In [ ]:
opt_times

In [ ]:
unopt_times

In [ ]:
comp=[unopt_times,opt_times]


In [ ]:
X = ['Log','KNN','SVC','DTC']



In [ ]:
import matplotlib.pyplot as plt
left = [1,2]
pred_times = [finish_opt - start_opt, finish_unopt - start_unopt]
tick_label = ['Intel Extension for SKLearn', 'Unoptimized SKLearn']
plt.bar(left, pred_times, tick_label = tick_label, width = 0.5, color = ['blue', 'red'])
plt.xlabel('Training Method'); plt.ylabel('time,s'); plt.title('Logistic Regression Training Time before standardizations')
plt.show()
print("Performance Speedup: ",(finish_unopt - start_unopt)/(finish_opt - start_opt))




In [ ]:
fig4=plt.bar(left, pred_times, tick_label = tick_label, width = 0.5, color = ['blue', 'red'])

In [ ]:
fig4

In [ ]:
pred_times_knn = [finish_opt_knn - start_opt_knn, finish_unopt_knn - start_unopt_knn]

In [ ]:
pred_times_svc=[finish_opt_svc - start_opt_svc, finish_unopt_svc - start_unopt_svc]

In [ ]:
pred_times_dtc=[finish_opt_dtc - start_opt_dtc, finish_unopt_dtc - start_unopt_dtc]

In [ ]:
plt.figure(figsize=(10, 8))
fig=plt.figure(figsize=(10, 8))
plt.subplot(2, 2, 1)
plt.bar(left, pred_times, tick_label = tick_label, width = 0.5, color = ['blue', 'red'])
plt.xlabel('Training Method'); plt.ylabel('time,s'); plt.title('Logistic Regression Comparision')


plt.subplot(2, 2, 2)
plt.bar(left, pred_times_knn, tick_label = tick_label, width = 0.5, color = ['blue', 'red'])
plt.xlabel('Training Method'); plt.ylabel('time,s'); plt.title('KNN Comparision')
plt.show()


plt.figure(figsize=(10, 8))
fig2=plt.figure(figsize=(10, 8))
plt.subplot(2, 2, 1)
plt.bar(left, pred_times_svc, tick_label = tick_label, width = 0.5, color = ['blue', 'red'])
plt.xlabel('Training Method'); plt.ylabel('time,s'); plt.title('SVC Comparision')
plt.show()



plt.figure(figsize=(10, 8))
fig3=plt.figure(figsize=(10, 8))
plt.subplot(2, 2, 2)
plt.bar(left, pred_times_dtc, tick_label = tick_label, width = 0.5, color = ['blue', 'red'])
plt.xlabel('Training Method'); plt.ylabel('time,s'); plt.title('Decision Tree Comparision')
plt.show()





In [ ]:
fig.savefig('full_figure.png')
fig2.savefig('full_figures.png')
fig3.savefig('full_figured.png')